In [2]:
from tqdm import tqdm
import librosa as lb
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt



import os
import pickle as pkl
from keras.utils import to_categorical

from sklearn.model_selection import train_test_split


In [3]:


feature_types = ['mfcc', 'lfcc', 'spectrogram', 'stft']
feature_type = 'stft'

model_checkpoint_filepath = './models/mfcc_ann.h5'

training_history_filepath = "./models/mfcc_ann_train_history.pkl"

extracted_data_filepath = "./feature_extraction_data/mfcc_USD_Folds.pkl"

re_train = True #Train Model Again

re_feature_extract = False #Feature Extract Again



In [4]:
EPOCHS = 100
BATCH_SIZE = 64

In [5]:
df = pd.read_csv('UrbanSound8K.csv')

In [9]:
def spectrograms_feature_extractor(path):
    data, sample_rate = lb.load(path, res_type='kaiser_fast')
    spectrogram = np.abs(lb.stft(data, n_fft=128))**2           #diff is in the power to 2, amplifies higher magintude values and suppresses lower ones
    spectrogram_features = np.mean(spectrogram, axis=1)
    return spectrogram_features

def lfcc_feature_extractor(path):
    data, sample_rate = lb.load(path, res_type='kaiser_fast')
    lfcc = lb.feature.lfcc(data, n_mfcc=128)
    lfcc_features = np.mean(lfcc, axis=1)
    return lfcc_features

def mfcc_feature_extractor(path):
    data, simple_rate = lb.load(path, res_type='kaiser_fast')
    data = lb.feature.mfcc(data, n_mfcc=128)
    data = np.mean(data,axis=1)
    return data

def stfts_feature_extractor(path):
    data, sample_rate = lb.load(path, res_type='kaiser_fast')
    stft = np.abs(lb.stft(data, n_fft=128))
    stft_features = np.mean(stft, axis=1)
    return stft_features



lfcc_x, lfcc_y = [], []
stft_x, stft_y = [], []
spec_x, spec_y = [], []

for i,rows in tqdm(df.iterrows()):
    path = '../data/UrbanSoundDatabaseFolds/' + 'fold' + str(rows['fold']) + '/' + str(rows['slice_file_name'])
    
    
    lfcc_x.append(lfcc_feature_extractor(path))
    lfcc_y.append(rows['classID'])
    
    stft_x.append(stft_feature_extractor(path))
    stft_y.append(rows['classID'])
    
    spec_x.append(spectrogram_feature_extractor(path))
    spec_y.append(rows['classID'])
    
    
        
lfcc_x = np.array(lfcc_x)
lfcc_y = np.array(lfcc_y)
lfcc_y = to_categorical(lfcc_y)

stft_x = np.array(stft_x)
stft_y = np.array(stft_y)
stft_y = to_categorical(stft_y)

spec_x = np.array(spec_x)
spec_y = np.array(spec_y)
spec_y = to_categorical(spec_y)
    
    
#saving to pickle
with open('./feature_extraction_data/lfcc_USD_Folds.pkl', 'wb') as file:
    pkl.dump((lfcc_x, lfcc_y), file)
    
with open('./feature_extraction_data/stft_USD_Folds.pkl', 'wb') as file:
    pkl.dump((stft_x, stft_y), file)
    
with open('./feature_extraction_data/spec_USD_Folds.pkl', 'wb') as file:
    pkl.dump((spec_x, spec_y), file)

0it [00:00, ?it/s]


AttributeError: module 'librosa.feature' has no attribute 'lfcc'

In [ ]:
from keras.layers import Dense, Activation, Flatten, Dropout, BatchNormalization
from keras.models import Sequential, Model
from keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras import regularizers, optimizers
from keras.callbacks import ModelCheckpoint
from keras.models import load_model

In [ ]:
from sklearn.metrics import confusion_matrix, plot_confusion_matrix
import matplotlib.pyplot as plt
from sklearn.metrics import f1_score